# api calls tests

> check the api calls are working as expected

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

## test module loads environment variables

In [ ]:
from assign_uprn.api_calls import AssignAPIClient
client = AssignAPIClient(dotenv_path='./.env')
assert isinstance(client, AssignAPIClient)

## test single address check

In [ ]:
comparable = {'Address_format': 'good',
 'Postcode_quality': 'good',
 'Matched': True,
 'BestMatch': {'UPRN': '100023336956',
  'Qualifier': 'Property',
  'LogicalStatus': '1',
  'Classification': 'RD04',
  'ClassTerm': 'Terraced',
  'Algorithm': '10-match1',
  'ABPAddress': {'Number': '10',
   'Street': 'Downing Street',
   'Town': 'City Of Westminster',
   'Postcode': 'SW1A 2AA'},
  'Match_pattern': {'Postcode': 'equivalent',
   'Street': 'equivalent',
   'Number': 'equivalent',
   'Building': 'equivalent',
   'Flat': 'equivalent'}}}

response = client.address_search('10 Downing St,Westminster,London,SW1A2AA')
assert response.json() == comparable

## test upload an encrypted salt

In [ ]:
comparable = {"upload": { "status": "SALTOK"}}

assert client.upload(infilepath="../data/external/test.EncryptedSalt").json() == comparable

## test upload a multiple address check file

Example file content:

```tsv
1 ⭾ 10 Downing St,Westminster,London,SW1A2AA
2 ⭾ Bridge Street,London,SW1A 2LW
3 ⭾ 221b Baker St,Marylebone,London,NW1 6XE
4 ⭾ 3 Abbey Rd,St Johns Wood,London,NW8 9AY
```


In [ ]:
comparable = {'upload': {'status': 'OK'}}

assert client.upload(infilepath="../data/external/test-addresses.txt").json() == comparable

## test download a multiple address match file


In [ ]:
import time
# GIVE TIME FOR THE UPLOAD TO BE PROCESSED DURING TESTS
time.sleep(10)

infilepath = '../data/external/test-addresses.txt'
outfilepath = '../data/processed/assign-uprn.tsv'
assert client.download(infilepath=infilepath, outfilepath=outfilepath).status_code == 200

written to ../data/processed/assign-uprn.tsv


The download will produce an output similar to the following: 

<div style="overflow-x: scroll">
| id | uprn | address_fmt | algorithm | classification | match_building | match_flat | match_number | match_postcode | match_street | abp_number | abp_postcode | abp_street | abp_town | qualifier | adr_candiddate | abp_building | latitude | longitude | point | x | y | ralf | classification_term | abp_flat | logical_status |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 1 | 100023336956 |  | 10-match1 | RD04 | equivalent | equivalent | equivalent | equivalent | equivalent | 10 | SW1A 2AA | Downing Street | City Of Westminster | Property | 10 Downing St,Westminster,London,SW1A2AA |  | 51.5035410 | -.1276700 | 51.5035410 | 530047.00 | 179951.00 | C30921C8404087803C3687301351FF41CCB4A5E8F3691070723293C8BD654CBB | Terraced |  | 1 |
| 2 | 200002501505 |  | 550-match5a | PP | candidate field dropped | equivalent | equivalent | equivalent | equivalent |  | SW1A 2LW | Bridge Street | City Of Westminster | Property | Bridge Street,London,SW1A 2LW | Portcullis House | 51.5013476 | -.1243451 | 51.5013476 | 530284.00 | 179713.00 | 4D19E2EB66A2C12BD56B93D96CFBBE5B74525AEFC4C68329BE87B55C43EA4C36 | Property Shell |  | 1 |
| 3 | 100023071949 |  | 3200-match61A170 | CR08 | moved from Number  | equivalent | moved to Building  | equivalent | equivalent |  | NW1 6XE | Baker Street | London | Property | 221b Baker St,Marylebone,London,NW1 6XE | 221B | 51.5237510 | -.1585550 | 51.5237510 | 527847.00 | 182144.00 | 7727B90C7C3A744AF6FD8D5A4FEB6767B1EACBBC721B85EED6AE86EDD2B0BA9C | Shop / Showroom |  | 1 |
| 4 | 100023122909 |  | 40-match1 | CR08 | moved from Street  | moved from Number  | moved from Flat  | equivalent | moved from Building  | 3 | NW8 9AY | Abbey Road | City Of Westminster | Property | 3 Abbey Rd,St Johns Wood,London,NW8 9AY |  | 51.5321562 | -.1779541 | 51.5321562 | 526478.00 | 183045.00 | 6E479D3F8DA8A548C631622EA8640E1CE9030289C5ED4458B91A4F6C4F92C799 | Shop / Showroom |  | 1 |
</div>
